In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import re

In [2]:
date = ['2025/07/16']
venue = ['HV', 'ST']

params = {
    'dates': date,
    'venues': venue
}

In [5]:
def fetch_race_data_by_venue(date, venues, max_race_no=9):
 
    base_url = "https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx"
    results = {}

    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; HKJCScraper/1.0)"
    }

    for venue in venues:
        print(f"Checking venue {venue} race no 1...")
        params = {
            "RaceDate": date,
            "Racecourse": venue,
            "RaceNo": 1
        }

        response = requests.get(base_url, params=params, headers=headers)
        if response.status_code != 200:
            print(f"Failed to fetch page for date {date} venue {venue} race 1, status: {response.status_code}")
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        # Check for the presence of the race result table, often with class 'table_bd' or id
        table = soup.find('table', class_='table_bd')

        if not table:
            print(f"No race data found for date {date} venue {venue} race 1; skipping rest of races for this venue.")
            continue  # Skip race 2-9 for this venue

        # Save race 1 result
        results[(venue, 1)] = soup
        print(f"Race data found for date {date} venue {venue} race 1; fetching races 2 to {max_race_no}...")

        # Now fetch the rest of the races for this venue
        for race_no in range(2, max_race_no + 1):
            params["RaceNo"] = race_no
            response = requests.get(base_url, params=params, headers=headers)
            if response.status_code != 200:
                print(f"Failed to fetch page for date {date} venue {venue} race {race_no}, status: {response.status_code}")
                results[(venue, race_no)] = None
                continue

            soup = BeautifulSoup(response.text, "html.parser")
            table = soup.find('table', class_='table_bd')
            if not table:
                print(f"No race data found for date {date} venue {venue} race {race_no}.")
                results[(venue, race_no)] = None
            else:
                results[(venue, race_no)] = soup

    return results


In [6]:
race_date = '2025/07/16'
venue_list = ['HV', 'ST']

data = fetch_race_data_by_venue(race_date, venue_list)
# 'data' contains BeautifulSoup objects for successful pages or None entries for misses

Checking venue HV race no 1...
Race data found for date 2025/07/16 venue HV race 1; fetching races 2 to 9...
Checking venue ST race no 1...
No race data found for date 2025/07/16 venue ST race 1; skipping rest of races for this venue.


In [ ]:
soup = BeautifulSoup(data.content, 'html.parser')

AttributeError: 'dict' object has no attribute 'content'